In [1]:
import tensorflow as tf
import numpy as np

In [1]:
'''
한개의 time 단위로 train => time 기준 batch_size=1
masking 사용하지 않음 => F1 score나 confusion matrix로 보여줄 것이기 때문
'''
class GCN_WITH_LSTM():
    def __init__(self, num_frame, nomalized_adj, input_dim=8, num_nodes=8, num_classes=4):

        init = tf.initializers.he_normal()
        lstm_hidden = 64
        num_lstm_layers = 1
        gcn_hidden = 64
        
        # placeholders
        self.x = tf.placeholder(tf.float32, [num_nodes, num_frame, input_dim])
        self.y = tf.placeholder(tf.float32, [num_nodes, num_classes])
        
        # normalized adj matrix for GCN
        # you can improve this by using sparse tensor when adj matrix is large
        self.A = tf.constant(nomalized_adj)

        cell = tf.nn.rnn_cell.LSTMCell(lstm_hidden)
        cell = tf.nn.rnn_cell.MultiRNNCell([cell] * num_layers)
        
        # forward lstm
        outputs, _ = tf.nn.rnn(cell, self.x) # outputs.shape: (num_nodes, lstm_hidden)
        
        # forward GCN
        self.w1 = tf.Variable(init, [lstm_hidden, gcn_hidden])
        self.h1 = tf.matmul(tf.matmul(self.A, outputs), self.w1)
        
        self.w2 = tf.Variable(init, [gcn_hidden, num_classes])
        self.h2 = tf.matmul(tf.matmul(self.A, self.h1), self.w2)
        
        # prediction
        self.pred = tf.argmax(self.h2, axis=1)
        
        # loss function
        # (as far as I know) we cannot directly optimize metric(F1 score, accuracy ...etc)
        self.loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.h2, labels=self.y))

        # optimizer
        optimizer = tf.train.AdamOptimizer()
        self.train_step = optimizer.minimize(self.loss)
        
        # metric - accuracy
        correct = tf.equal(tf.argmax(self.h2, axis=1), tf.argmax(self.y, axis=1))
        self.accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [2]:
def get_adj_matrix():
    return [
        [0, 1, 0, 0, 0, 0, 0, 1],
        [1, 0, 1, 0, 0, 0, 0, 0],
        [0, 1, 0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 1, 0, 1],
        [1, 0, 0, 0, 0, 0, 1, 0],
    ]
    
# make DAD(normalization) matrix
def normalize_matrix(adj_matrix):
    a_tilda = adj_matrix + np.eye(adj_matrix.shape[0]) # A_ = A+I
    d_tilda = np.diag(1 / np.sqrt(np.sum(a_tilda, axis=1))) # D_^(-1/2)
    return np.matmul(np.matmul(d_tilda, a_tilda), d_tilda)

In [ ]:
def train_model():
    epochs = 300
    normalized_adj = normalize_matrix(get_adj_matrix())
    frame_size = 
    after = 
    
    
    ##########################
    data = load_data() # shape: (num_nodes, 9, 320000)
    x, y = preprocess_data()
    x.shape = (320000-frame_size-after+1, num_nodes, 8)
    y.shape = (320000-frame_size-after+1, num_nodes)
    
    for t in range(320000-frame_size-after+1):
        curr_x = x[t, :, :]
        curr_y = y[t, :, :]
    ##########################